In [14]:
import glob
import os
path = 'train/chinese_news_trans/'
test_path = 'test/chinese_news_test/'
# print(glob.glob('./train/chinese_news_trans/*'))
trans = {os.path.basename(name):index for index,name in enumerate(glob.glob(path+'*'))}

In [22]:
import pandas as pd
dic = []
test_dic = []
for k in trans.keys():
    print(k)
    all_files = glob.glob(f'{path}{k}/*.txt')
    test_files = glob.glob(f'{test_path}{k}/*.txt')
    for i in all_files:
#         print(i)
        with open(i,'r', encoding='UTF-8') as f:
            output = f.read()
            dic.append({'category':trans[k],'content':output.split('【 正  文 】')[-1]})
            
    for i in test_files:
#         print(i)
        with open(i,'r', encoding='UTF-8') as f:
            output = f.read()
            test_dic.append({'category':trans[k],'content':output.split('【 正  文 】')[-1]})
            
print(dic)
train_df = pd.DataFrame(dic)
test_df = pd.DataFrame(test_dic)
# train_df
test_df

交通
政治
教育
環境
經濟
藝術
計算機
軍事
醫藥
體育


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



,category,content
0,0,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...
1,0,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...
2,0,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...
3,0,三趟火車停開 乘客可全額退票 \n瀏覽次數：1180 \n 昨日，來自鐵路部門的消息說...
4,0,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...
...,...,...
96,9,最優秀選手無緣亞運會健美賽\n \n \n 健美在亞洲運動會上是“新生兒”——韓國釜...
97,9,\n各國記者眼中的羽毛球世錦賽\n \n\n-----------------------...
98,9,\n友好運動會第五天 東道主選手大顯神威\n2001年09月03日02:51:40 新華社 ...
99,9,不靠技術比運氣\n\n 第二屆奧運會在巴黎舉行，同時這裡也正在舉行國際博覽會，東道主把一些...


In [3]:
from jieba import cut
def poemcut(s):
    s = (s.replace("\r", ' ')
      .replace("\n", ' ')
#       .replace("，", ' ')
#       .replace("。", ' ')
        )
    return ' '.join(cut(s))

In [4]:
x_train = train_df['content'].apply(poemcut)
y_train = train_df['category']
x_test = test_df['content'].apply(poemcut)
y_test = test_df['category']

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\User\AppData\Local\Temp\jieba.cache
Loading model cost 0.664 seconds.
Prefix dict has been built succesfully.


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_vec = vec.fit_transform(x_train)

In [6]:
reverse = {v:k for k,v in vec.vocabulary_.items()}

In [7]:

# 單純貝式分類器
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train_vec,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [8]:
# 測試資料不fit
x_test_vec = vec.transform(x_test)

In [9]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_vec)
accuracy_score(pre,y_test)

1.0

In [10]:
# 混淆矩陣
from sklearn.metrics import confusion_matrix
labels = [k for k in trans.keys()]
pd.DataFrame(confusion_matrix(y_test,pre),
             columns=[f'{l} (預測)' for l in labels],
             index=[f'{l} (正確)' for l in labels])

,交通 (預測),政治 (預測),教育 (預測),環境 (預測),經濟 (預測),藝術 (預測),計算機 (預測),軍事 (預測),醫藥 (預測),體育 (預測)
交通 (正確),10,0,0,0,0,0,0,0,0,0
政治 (正確),0,11,0,0,0,0,0,0,0,0
教育 (正確),0,0,10,0,0,0,0,0,0,0
環境 (正確),0,0,0,10,0,0,0,0,0,0
經濟 (正確),0,0,0,0,10,0,0,0,0,0
藝術 (正確),0,0,0,0,0,10,0,0,0,0
計算機 (正確),0,0,0,0,0,0,10,0,0,0
軍事 (正確),0,0,0,0,0,0,0,10,0,0
醫藥 (正確),0,0,0,0,0,0,0,0,10,0
體育 (正確),0,0,0,0,0,0,0,0,0,10


In [23]:
p = input('請寫一篇新聞')
p = poemcut(p)
p_vec = vec.transform([p])
print('應該是哪一類的->',labels[clf.predict(p_vec)[0]])

請寫一篇新聞〔記者陳柔蓁／台北報導〕國民黨今在三大報刊登半版廣告，喊話希望郭台銘回黨團結、尊重黨籍提名人制度，還附有31位大老簽名連署，外傳郭台銘是因為看到交情匪淺的前總統馬英九、國民黨副主席郝龍斌的簽名，才憤而在今日上午宣布退黨，對此郭辦回應，「此臆測並非事實」。  當初是馬英九勸進郭台銘參與國民黨初選，而國民黨內與郭台銘關係較和緩的，還有副主席郝龍斌，甚至扮演穿梭溝通的關鍵角色。  今日上午，郭幕僚、永齡基金會副執行長蔡沁瑜說，「退黨是郭台銘先生的個人決定和意志，跟馬前總統無關。郭台銘退黨，這是這麼長久以來，心境的表達。」  而昨日國民黨中常委對郭喊話「好聚好散」、「錢都已還清」，是否因此激怒郭台銘。今日蔡沁瑜對此表示，不對這件事回應。僅強調，現狀與郭台銘當初回國民黨的理念完全背道而馳，對於最後選擇退黨，表達遺憾、痛心。
應該是哪一類的-> 政治
